In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
import matplotlib.pyplot as plt
import numpy as np
import math
import time
from scipy.stats import randint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
excel_file1 = 'C:\\Users\\satya\\OneDrive\\Desktop\\cleans.xlsx'
df = pd.read_excel(excel_file1)
csv_file = 'stry.csv'  
df.to_csv('stry.csv', index=False)


# Label Encoding for categorical columns
label_encoder = LabelEncoder()
df['Payment Mode'] = label_encoder.fit_transform(df['Payment Mode'])
# Transform 'Segment', 'Region', 'Ship Mode' columns
df['Segment'] = label_encoder.fit_transform(df['Segment'])
df['Region'] = label_encoder.fit_transform(df['Region'])
df['Ship Mode'] = label_encoder.fit_transform(df['Ship Mode'])

df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m-%d-%Y')

df2019 = df[df['Order Date'].dt.year == 2019]
df2020 = df[df['Order Date'].dt.year == 2020]

X = df2019[['Region', 'Segment', 'Ship Mode','Quantity','Payment Mode']]
y = df2019['Sales']


df2020.drop('Sales',axis=1,inplace=True)
start_time_training = time.time()

# Assuming 'X' is the quantity feature and 'y' is the corresponding profits
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_trainre=y_train.values.reshape(-1,1)
y_testre=y_test.values.reshape(-1,1)
param_dist = {
    'n_estimators': randint(50, 500),  # Randomly sample between 50 to 500 estimators
    'max_depth': randint(2, 20),       # Randomly sample max depth between 2 to 20
    'min_samples_split': randint(2, 10), # Randomly sample min samples split between 2 to 10
    # Add other hyperparameters as needed
}


xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 100)

# Train the model
xg_reg.fit(X_train, y_trainre)



# Training the model

end_time_training = time.time()
training_time = end_time_training - start_time_training
useful=df2020[['Region', 'Segment', 'Ship Mode','Quantity','Payment Mode']]


start_time_predictions = time.time()# Making predictions
predictions = xg_reg.predict(X_test)

end_time_predictions = time.time()
prediction_time = end_time_predictions - start_time_predictions
mse = mean_squared_error(y_testre, predictions)
print(f"Mean Squared Error: {mse}")
print(math.sqrt(mse))

predictions1 = xg_reg.predict(useful)
print(predictions1)

print(f"Time taken for training: {training_time} seconds")
print(f"Time taken for predictions: {prediction_time} seconds")
x_values_scatter = df2019['Sales'].values
y_values_scatter = df2019['Quantity'].values
x_values_line = predictions1
y_values_line = df2020['Quantity'].values
# Create a figure and axis
fig, ax = plt.subplots()

# Plotting scatter plot
ax.scatter(x_values_scatter, y_values_scatter, label='Scatter', color='red')

# Plotting line plot
ax.plot(x_values_line, y_values_line, label='Line', color='blue')

# Set labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_title('Scatter and Line Plot')

# Show legend
ax.legend()

# Show the plot
plt.show()




Mean Squared Error: 333578.4253463249
577.5624860968074
[255.22342 252.2159  268.37418 ... 208.34566 197.19983 221.49658]
Time taken for training: 0.045273542404174805 seconds
Time taken for predictions: 0.04141807556152344 seconds


C:\Users\satya\AppData\Local\Temp\ipykernel_2756\1658517178.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020.drop('Sales',axis=1,inplace=True)
